In [1]:
%load_ext autoreload

In [2]:
import lightning.pytorch as pl
import torch
import pandas as pd
import os
import scanpy as sc
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt

In [18]:
%autoreload 2
from self_supervision.estimator.cellnet import EstimatorAutoEncoder
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.tester.reconstruction.test import eval_rec, test_pretrained_model, test_models_on_ood, test_random_model

In [4]:
DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/merlin_cxg_2023_05_15_sf-log1p'  # this is a shared directory in which the data is stored
MODEL_PATH = '/lustre/groups/ml01/workspace/till.richter/trained_models/'
RESULT_PATH = '/lustre/groups/ml01/workspace/till.richter/ssl_results'
HVG = False

### Initialize the lightning model

In [5]:
# init estim class
estim = EstimatorAutoEncoder(DATA_PATH, hvg=HVG)

In [6]:
# init datamodule
estim.init_datamodule(batch_size=1024) 

In [7]:
# init model
hidden_units = [512, 512, 256, 256, 64]
estim.init_model(
    model_type='mlp_ae',
    model_kwargs={
        'learning_rate': 1e-3,
        'weight_decay': 0.1,
        'lr_scheduler': torch.optim.lr_scheduler.StepLR,
        'lr_scheduler_kwargs': {
            'step_size': 2,
            'gamma': 0.9,
            'verbose': True
        },
        'units_encoder': hidden_units,
        'units_decoder': hidden_units[::-1][1:],
    },
)

In [8]:
estim.trainer = pl.Trainer(logger=[], accelerator='cpu', devices=1)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Load all models of interest

In [9]:
"""
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    # Individual Gene Masking
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50p/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gene_program_C8_25p/default/version_2/checkpoints/best_checkpoint_val.ckpt',

    # Gene Program to Gene Program
    MODEL_PATH + '/pretext_models/masking/CN_MLP_single_gene_program/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)
    # MODEL_PATH + '/pretext_models/contrastive/MLP_BYOL_Gaussian_0_001_v4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins
    # MODEL_PATH + '/pretext_models/contrastive/MLP_bt_Gaussian_0_001/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Individual Gene Masking
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_50prun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program Masking
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gene_program_C8_25prun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Gene Program    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_single_gene_programrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Boostrap Your Own Latent (BYOL)    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_MLP_BYOL_Gaussian_0_001run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    # Barlow Twins    
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_contrastive_MLP_bt_Gaussian_0_01run0/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]
"""

"\nmodel_dirs = [\n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    ### Only Pretrained Models ###\n    # Individual Gene Masking\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_50p/default/version_1/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_50prun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pr

In [9]:

# Negative Binomial Fitting
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

### Evaluate all models

In [10]:
metrics = eval_rec(estim, model_dirs)

Skipping model:  CN_No_SSL_CN_run0NegBin_
Skipping model:  CN_No_SSL_CN_run1NegBin_
Skipping model:  CN_No_SSL_CN_run2NegBin_
Skipping model:  CN_No_SSL_CN_run3NegBin_
Skipping model:  CN_No_SSL_CN_run4NegBin_
Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run0NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run1NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run2NegBin_
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run3NegBin_
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     0.1804896593093872      │
│ test_explained_var_weighted │     0.3797783851623535      │
│          test_loss          │      4683.22509765625       │
│          test_mse           │     0.10016700625419617     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0run4NegBin_  :
 [{'test_explained_var_uniform': 0.1804896593093872, 'test_explained_var_weighted': 0.3797783851623535, 'test_mse': 0.10016700625419617, 'test_loss': 4683.22509765625}]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    -0.025042209774255753    │
│ test_explained_var_weighted │   -0.0002877476217690855    │
│          test_loss          │       12265.205078125       │
│          test_mse           │     0.5372250080108643      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  Random  :
 [{'test_explained_var_uniform': -0.025042209774255753, 'test_explained_var_weighted': -0.0002877476217690855, 'test_mse': 0.5372250080108643, 'test_loss': 12265.205078125}]


# Fig 3: OOD Reconstruction

**Dissection: Tail of Hippocampus (HiT) - Caudal Hippocampus - CA4-DGC**

- 56,367 cells
- 10x 3' v3
- hippocampal formation
- astrocyte (3761), central nervous system macrophage (1782), endothelial cell (174), ependymal cell (111), ~~fibroblast (86)~~, leukocyte (36), neuron (36588), oligodendrocyte (11875), oligodendrocyte precursor cell (1896), pericyte (39), vascular associated smooth muscle cell (19)

In [24]:
ood_set = 'tail_of_hippocampus'

In [13]:
"""
model_dirs = [
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun0/default/version_3/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt', \n    \n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpo

In [25]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Evaluating model:  CN_No_SSL_CN_run0NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.021411335095763206     │
│ test_explained_var_weighted │     0.07335852086544037     │
│          test_loss          │        6099.49609375        │
│          test_mse           │     0.10462138056755066     │
└─────────────────────────────┴─────────────────────────────┘

Evaluating model:  CN_No_SSL_CN_run1NegBin_


...

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.022021697834134102     │
│ test_explained_var_weighted │     0.07405642420053482     │
│          test_loss          │      6097.37353515625       │
│          test_mse           │     0.10460340976715088     │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    -0.003548241686075926    │
│ test_explained_var_weighted │   -8.459998934995383e-05    │
│          test_loss          │      0.340019166469574      │
│          test_mse           │      0.340019166469574      │
└─────────────────────────────┴─────────────────────────────┘

**All non-neuronal cells**

- 888,263 cells
- 10x 3' v3
- Bergmann glial cell (8041), astrocyte (155025), central nervous system macrophage (91383), ~~choroid plexus epithelial cell (7689)~~, endothelial cell (5165), ependymal cell (5882), ~~fibroblast (9156)~~, oligodendrocyte (494966), oligodendrocyte precursor cell (105734), pericyte (3693), vascular associated smooth muscle cell (1074)

In [10]:
ood_set = 'non_neuronal'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

Evaluating model:  CN_No_SSL_CN_run0NegBin_


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.012459386140108109     │
│ test_explained_var_weighted │    0.039653677493333817     │
│          test_loss          │       5715.9228515625       │
│          test_mse           │     0.14177198708057404     │
└─────────────────────────────┴─────────────────────────────┘

Evaluating model:  CN_No_SSL_CN_run1NegBin_


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.012672662734985352     │
│ test_explained_var_weighted │     0.04026352986693382     │
│          test_loss          │       5713.365234375        │
│          test_mse           │     0.14166690409183502     │
└─────────────────────────────┴─────────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

In [ ]:
ood_set = 'cort_dev'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

In [ ]:
ood_set = 'circ_imm'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

In [ ]:
ood_set = 'great_apes'

In [ ]:
test_models_on_ood(estim, ood_set=ood_set, model_dirs=model_dirs)

# Fig 4: Individual Atlas

### HLCA

In [9]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__HLCA/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]

In [10]:
supervised_subset = 148  # HLCA

In [11]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005182110471650958    │
│ test_explained_var_weighted │   -6.305475108092651e-05    │
│          test_loss          │     0.35207226872444153     │
│          test_mse           │     0.35207226872444153     │
└─────────────────────────────┴─────────────────────────────┘

In [12]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run1NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run2NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run3NegBin__HLCA
Skipping model:  CN_No_SSL_CN_new_run4NegBin__HLCA
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__HLCA
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__HLCA
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.049650926142930984     │
│ test_explained_var_weighted │     0.2730991840362549      │
│          test_loss          │       4347.6298828125       │
│          test_mse           │     0.09400048106908798     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__HLCA  :
 [{'test_explained_var_uniform': 0.049650926142930984, 'test_explained_var_weighted': 0.2730991840362549, 'test_mse': 0.09400048106908798, 'test_loss': 4347.6298828125}]
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__HLCA


...

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__HLCA  :
 [{'test_explained_var_uniform': 0.0478813536465168, 'test_explained_var_weighted': 0.2738361358642578, 'test_mse': 0.09390199184417725, 'test_loss': 4345.16650390625}]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     -0.0253374632447958     │
│ test_explained_var_weighted │   -0.00030291362782008946   │
│          test_loss          │      12285.4345703125       │
│          test_mse           │     0.5380129814147949      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  Random  :
 [{'test_explained_var_uniform': -0.0253374632447958, 'test_explained_var_weighted': -0.00030291362782008946, 'test_mse': 0.5380129814147949, 'test_loss': 12285.4345703125}]


### PBMC

In [13]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    \n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_PBMC/default/version_0/checkpoints/best_check

In [19]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt', 
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__PBMC/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run5NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run6NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run7NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run5NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run6NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [20]:
supervised_subset = 41  # PBMC

In [13]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -2.058938298432622e-05    │
│ test_explained_var_weighted │   -2.823654722305946e-05    │
│          test_loss          │     0.37847262620925903     │
│          test_mse           │     0.37847262620925903     │
└─────────────────────────────┴─────────────────────────────┘

In [21]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run1NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run2NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run3NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run4NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run5NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run6NegBin__PBMC
Skipping model:  CN_No_SSL_CN_new_run7NegBin__PBMC
Evaluating model:  CN_No_SSL_CN_new_run8NegBin__PBMC
Could not load model:  CN_No_SSL_CN_new_run8NegBin__PBMC
[Errno 2] No such file or directory: '/lustre/groups/ml01/workspace/till.richter/trained_models//final_models/reconstruction/CN_No_SSL_CN_new_run8NegBin__PBMC/default/version_0/checkpoints/best_checkpoint_val.ckpt'
Could not load model:  CN_No_SSL_CN_new_run8NegBin__PBMC
update_wei

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.024820495396852493     │
│ test_explained_var_weighted │     0.13843071460723877     │
│          test_loss          │      3710.58740234375       │
│          test_mse           │     0.11273898929357529     │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run5NegBin__PBMC  :
 [{'test_explained_var_uniform': 0.024820495396852493, 'test_explained_var_weighted': 0.13843071460723877, 'test_mse': 0.11273898929357529, 'test_loss': 3710.58740234375}]
Evaluating model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run6NegBin__PBMC


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │    0.024853991344571114     │
│ test_explained_var_weighted │     0.13898102939128876     │
│          test_loss          │       3709.5068359375       │
│          test_mse           │     0.1126665323972702      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run6NegBin__PBMC  :
 [{'test_explained_var_uniform': 0.024853991344571114, 'test_explained_var_weighted': 0.13898102939128876, 'test_mse': 0.1126665323972702, 'test_loss': 3709.5068359375}]


### Tabula Sapiens

In [18]:
"""
model_dirs = [
    ### Only Pretrained Models ###
    
    # Gene Program to Transcription Factor    
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    # Gene Program to Transcription Factor
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run1_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run2_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run3_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_MLP_gp_to_tfnew_run4_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
]
"""

"\nmodel_dirs = [\n    ### Only Pretrained Models ###\n    \n    # Gene Program to Transcription Factor    \n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tf/default/version_2/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun1/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun2/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun3/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/pretext_models/masking/CN_MLP_gp_to_tfrun4/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    \n    \n    ### Supervised Models ###\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run0_Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',\n    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_MLPnew_run1_Tabula_Sapiens/default/version_0/ch

In [22]:
model_dirs = [
    ### Only Pretrained Models ###
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun0/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun1/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun2/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun3/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/pretext_models/masking/CN_NegBin_gp_to_tfrun4/default/version_1/checkpoints/best_checkpoint_val.ckpt',
    
    
    ### Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_No_SSL_CN_new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    
    ### Self-Supervised Models ###
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
    MODEL_PATH + '/final_models/reconstruction/CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__Tabula_Sapiens/default/version_0/checkpoints/best_checkpoint_val.ckpt',
]

In [23]:
supervised_subset = 87  # Tabula Sapiens

In [21]:
metrics = test_random_model(estim, supervised_subset=supervised_subset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │   -0.0005711353733204305    │
│ test_explained_var_weighted │   -6.664633110631257e-05    │
│          test_loss          │     0.3395370543003082      │
│          test_mse           │     0.3395370543003082      │
└─────────────────────────────┴─────────────────────────────┘

In [24]:
metrics = eval_rec(estim, model_dirs, supervised_subset=supervised_subset)

Skipping model:  CN_NegBin_gp_to_tfrun0
Skipping model:  CN_NegBin_gp_to_tfrun1
Skipping model:  CN_NegBin_gp_to_tfrun2
Skipping model:  CN_NegBin_gp_to_tfrun3
Skipping model:  CN_NegBin_gp_to_tfrun4
Skipping model:  CN_No_SSL_CN_new_run0NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run1NegBin__Tabula_Sapiens
Skipping model:  CN_No_SSL_CN_new_run2NegBin__Tabula_Sapiens
Evaluating model:  CN_No_SSL_CN_new_run3NegBin__Tabula_Sapiens


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test_explained_var_uniform  │     0.03977331519126892     │
│ test_explained_var_weighted │     0.25627702474594116     │
│          test_loss          │       4909.5283203125       │
│          test_mse           │      0.086342453956604      │
└─────────────────────────────┴─────────────────────────────┘

Metrics for model:  CN_No_SSL_CN_new_run4NegBin__Tabula_Sapiens  :
 [{'test_explained_var_uniform': 0.03977331519126892, 'test_explained_var_weighted': 0.25627702474594116, 'test_mse': 0.086342453956604, 'test_loss': 4909.5283203125}]
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run0NegBin__Tabula_Sapiens
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run1NegBin__Tabula_Sapiens
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run2NegBin__Tabula_Sapiens
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run3NegBin__Tabula_Sapiens
Skipping model:  CN_SSL_CN_CN_NegBin_gp_to_tfrun0new_run4NegBin__Tabula_Sapiens
